In [ ]:
from MESAreader import getSrcCol
from lib_plot_bin import plot_ratio_BE_r, get_ax_from_pfile, Rsun_cm
from lib_engineered import get_M_boundary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import warnings
import paths

In [ ]:
root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/"
root_rot = root + "single_stars/Z_0.0019/"
folders_rot = sorted(glob.glob(root_rot+"/18_rot0*/"))
accretor_root = "/mnt/home/mrenzo/ceph/RUNS/accretors/mesa15140/binaries/Z_0.0019/m1_18.0000_m2_15.0000_initial_z_0.0019_initial_period_in_days_1.0000e+02_grid_index_0_1"

In [ ]:
def get_rot_from_folder(f):
    """
    Parameters:
    -----------
    f: `string` or `path` for the folder. The
       folder name has to be formated as arbitrary/path/ending/in/*_rot/
    Returns:
    -------
    float with initial rotation rate assumed for the star.
    """
    return float(f.split('/')[-2].split('_rot')[-1])


In [ ]:
colors = plt.cm.viridis(np.linspace(0, 1, len(folders_rot)))

In [ ]:
def plot_one_outer_R(pfile_accretor, ax, folders_rot, radius_string, legend=False):
    ratio = plot_ratio_BE_r(
        pfile_accretor,
        pfile_accretor,
        ax,
        alpha_th=1.0,
        alpha_rot=0.0,
        color="orange",
        ls="-",
        lw=2,
        zorder=0)
    # highlight CEB boundaries for accretor
    delta_M_boundary, max_M_boundary, min_M_boundary = get_M_boundary(pfile_accretor, offset=0.01)
    src, col = getSrcCol(pfile_accretor)
    m = src[:, col.index("mass")]
    r = src[:, col.index("radius")]
    r_inner = r[np.argmin(np.absolute(m-min_M_boundary))]
    r_outer = r[np.argmin(np.absolute(m-max_M_boundary))]
    ax.axvspan(np.log10(r_inner*Rsun_cm), np.log10(r_outer*Rsun_cm), fc="#808080", alpha=0.5, zorder=0)
    
    for f in folders_rot:
        p = glob.glob(f + "/LOGS/"+radius_string)[0]
        label = "$\omega/\omega_\mathrm{crit}="+str(get_rot_from_folder(f))+"$"
        if "rot0.0" in f:
            c = 'r'
            lw=3
            z=10
        else:
            c=colors[folders_rot.index(f)]
            lw=2
            z=3
        ratio = plot_ratio_BE_r(
            p,
            pfile_accretor,
            ax,
            alpha_th=1.0,
            alpha_rot=0.0,
            color=c,
            ls="-",
            lw=lw,
            zorder=z,
            label=label)
        ax.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")
        ax.set_ylabel(r"$BE(\mathrm{accretor})/BE(\mathrm{single})$")
        if legend: ax.legend(ncol=2)


In [ ]:
# fig = plt.figure(figsize=(15,15))
# gs = gridspec.GridSpec(100, 100)
fig = plt.figure(figsize=(16, 24))
gs = gridspec.GridSpec(120, 150)
ax1 = fig.add_subplot(gs[:20, :50])
ax2 = fig.add_subplot(gs[20:40, :50])
ax3 = fig.add_subplot(gs[40:60, :50])
ax4 = fig.add_subplot(gs[60:80, :50])
ax5 = fig.add_subplot(gs[80:100, :50])

axes = [ax1, ax2, ax3, ax4, ax5]
for ax in axes:
    ax.set_xlim(8.2, 14)
    ax.set_ylim(-0.05, 2.5)
    if ax != axes[-1]:
        ax.set_xticklabels([])
        # ax.axhline(1, 0, 1, lw=2, ls="--", c="#808080")
        ax.text(
            0.05,
            0.85,
            "accretor more bound",
            fontsize=20,
            transform=ax.transAxes,
            va="bottom",
            ha="left",
            zorder=1,
        )
        ax.text(
            0.05,
            0.05,
            "accretor less bound",
            fontsize=20,
            transform=ax.transAxes,
            va="bottom",
            ha="left",
            zorder=1,
        )
    else:
        ax.set_xlabel(r"$\log_{10}(r/\mathrm{[cm]})$")
    ax3.set_ylabel(r"$BE(\mathrm{accretor})/BE(\mathrm{single})$")
    
    accretor_profiles = sorted(glob.glob(accretor_root + "/LOGS2/" + "*Rsun.data"))
    for pfile_accretor in accretor_profiles:
        string = pfile_accretor.split("/")[-1]
        if string == "20Rsun.data": continue
        ax = get_ax_from_pfile(pfile_accretor, axes)
        if ax == ax1:
            legend = True
        else:
            legend = False
        plot_one_outer_R(pfile_accretor, ax, folders_rot, string, legend=legend)
